## Code for cs5785 final, Team Ground Truth
### 2. Model part
- Author:  Hongliang CHI, hc962@cornell.edu,Kai Zhang kz298@cornell.edu

- Please make sure to have generated Resent features in the fold that this file in; The augmented Resent features can be generated by running "extract_resnet_features OR download through Google drive. Please read README for further details

In [1]:
import os
import csv
import random
import gensim
import numpy as np
from scipy.spatial.distance import cosine
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import re
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk.stem.porter import PorterStemmer  
import warnings
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
warnings.simplefilter("ignore")
from sklearn.externals import joblib
import gc
from sklearn.kernel_ridge import KernelRidge

In [2]:
# set the train,validation and testing set index
num_train = 8000
num_dev = 2000
num_test = 2000
split_idx = list(range(num_train + num_dev))

### Helper functions

In [3]:
punctuation = '!.,;:?"\''
def removePunctuation(text):
    text = re.sub(r'[{}]+'.format(punctuation),'',text)
    return text.strip().lower()

def parse_descriptions(data_dir, num_doc):
    docs = []
    for i in range(num_doc):
        path = os.path.join(data_dir, "%d.txt" % i)
        with open(path) as f:
            docs.append(f.read())
    return docs

def parse_features(features_path):
    vec_map = {}
    with open(features_path) as f:
        for row in csv.reader(f):
            img_id = int(row[0].split("/")[1].split(".")[0])
            vec_map[img_id] = np.array([float(x) for x in row[1:]])
    return np.array([v for k, v in sorted(vec_map.items())])

def doc_to_vec(sentence, word2vec):
    # get list of word vectors in sentence
    stop_words = set(['all', 'just', 'being', 'over', 'both', 'through', 'yourselves', 'its',
'before', 'herself', 'had', 'should', 'to', 'only', 'under', 'ours', 'has', 'do',
'them', 'his', 'very', 'they', 'not', 'during', 'now', 'him', 'nor', 'did', 'this',
'she', 'each', 'further', 'where', 'few', 'because', 'doing', 'some', 'are', 'our',
'ourselves', 'out', 'what', 'for', 'while', 'does', 'above', 'between', 't', 'be',
'we', 'who', 'were', 'here', 'hers', 'by', 'on', 'about', 'of', 'against', 's', 'or',
'own', 'into', 'yourself', 'down', 'your', 'from', 'her', 'their', 'there', 'been',
'whom', 'too', 'themselves', 'was', 'until', 'more', 'himself', 'that', 'but', 'don',
'with', 'than', 'those', 'he', 'me', 'myself', 'these', 'up', 'will', 'below', 'can',
'theirs', 'my', 'and', 'then', 'is', 'am', 'it', 'an', 'as', 'itself', 'at', 'have',
'in', 'any', 'if', 'again', 'no', 'when', 'same', 'how', 'other', 'which', 'you',
'after', 'most', 'such', 'why', 'a', 'off', 'i', 'yours', 'so', 'the', 'having',
'once'])
    word_tokens = word_tokenize(removePunctuation(sentence))
    filtered_sentence = list(set([w for w in word_tokens if not w in stop_words]))
    word_vecs = [word2vec.get_vector(w) for w in filtered_sentence if w in word2vec.wv.vocab]
    if len(word_vecs) == 0 :
        return np.zeros(300)
    return np.stack(word_vecs).mean(0)

def sentence_filter(sentence):
      # text cleansing for Bag of words BOW
    lemma = nltk.wordnet.WordNetLemmatizer()
    stop_words = set(['all', 'just', 'being', 'over', 'both', 'through', 'yourselves', 'its',
'before', 'herself', 'had', 'should', 'to', 'only', 'under', 'ours', 'has', 'do',
'them', 'his', 'very', 'they', 'not', 'during', 'now', 'him', 'nor', 'did', 'this',
'she', 'each', 'further', 'where', 'few', 'because', 'doing', 'some', 'are', 'our',
'ourselves', 'out', 'what', 'for', 'while', 'does', 'above', 'between', 't', 'be',
'we', 'who', 'were', 'here', 'hers', 'by', 'on', 'about', 'of', 'against', 's', 'or',
'own', 'into', 'yourself', 'down', 'your', 'from', 'her', 'their', 'there', 'been',
'whom', 'too', 'themselves', 'was', 'until', 'more', 'himself', 'that', 'but', 'don',
'with', 'than', 'those', 'he', 'me', 'myself', 'these', 'up', 'will', 'below', 'can',
'theirs', 'my', 'and', 'then', 'is', 'am', 'it', 'an', 'as', 'itself', 'at', 'have',
'in', 'any', 'if', 'again', 'no', 'when', 'same', 'how', 'other', 'which', 'you',
'after', 'most', 'such', 'why', 'a', 'off', 'i', 'yours', 'so', 'the', 'having',
'once'])
    word_tokens = word_tokenize(removePunctuation(sentence))
    filtered_sentence = list(set([w for w in word_tokens if not w in stop_words]))
    filtered_sentence = filtered_sentence + [lemma.lemmatize(i) for i in filtered_sentence]
    # there may be empty 
    if len(sentence) == 0 :
        return ''
    return ' '.join(filtered_sentence)

def dist_matrix(x1, x2):
    #cosine distance
    return np.array([cosine(i,j ) for i in x1 for j in x2]).reshape(2000,2000)

### Data Read and Feature Engineering 

In [4]:
#read in Google word2vec model
word2vec = gensim.models.KeyedVectors.\
            load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

In [5]:
# read in word2vec description model
train_dev_desc = parse_descriptions("./descriptions_train", num_doc=(num_train+num_dev))
test_desc = parse_descriptions("./descriptions_test", num_doc=num_test)
x_train_vec = np.array([doc_to_vec(train_dev_desc[i], word2vec) for i in split_idx[:num_train]])
x_dev_vec = np.array([doc_to_vec(train_dev_desc[i], word2vec) for i in split_idx[num_train:]])
x_test_vec = np.array([doc_to_vec(d, word2vec) for d in test_desc])

print("Built all word2vec description!")
print("x_train_vec shape:", x_train_vec.shape)
print("x_dev_vec shape:", x_dev_vec.shape)
print("x_test_vec shape:", x_test_vec.shape)

Built all word2vec description!
x_train_vec shape: (8000, 300)
x_dev_vec shape: (2000, 300)
x_test_vec shape: (2000, 300)


In [6]:
# read in word2vec tags model
train_dev_tag = parse_descriptions("./tags_train", num_doc=(num_train+num_dev))
test_tag = parse_descriptions("./tags_test", num_doc=num_test)
x_train_tag = np.array([doc_to_vec(train_dev_tag[i], word2vec) for i in split_idx[:num_train]])
x_dev_tag = np.array([doc_to_vec(train_dev_tag[i], word2vec) for i in split_idx[num_train:]])
x_test_tag = np.array([doc_to_vec(d, word2vec) for d in test_tag])

print("Built all word2vec tags!")
print("x_train_tag shape:", x_train_tag.shape)
print("x_dev_tag shape:", x_dev_tag.shape)
print("x_test_tag shape:", x_test_tag.shape)

Built all word2vec tags!
x_train_tag shape: (8000, 300)
x_dev_tag shape: (2000, 300)
x_test_tag shape: (2000, 300)


In [7]:
# read in BOW description model
vectorizer = CountVectorizer(min_df= 1,max_df = 0.50)
X = vectorizer.fit_transform([sentence_filter(d) for d in train_dev_desc+ test_desc]).toarray()
#X_train_bow = vectorizer.transform(np.array([sentence_filter(train_dev_desc[i]) for i in split_idx])).toarray()
#train test split
x_train_bow = X [:8000,:]
x_dev_bow =X[8000:10000,:]
X_test_bow = X [10000:,:]
# for submission output
x_train_all_bow = x_train_bow

print("Built all BOW description!")
print("x_train_bow shape:", x_train_bow.shape)
print("x_dev_bow shape:", x_dev_bow.shape)
print("X_test_bow  shape:",X_test_bow .shape)

Built all BOW description!
x_train_bow shape: (8000, 10634)
x_dev_bow shape: (2000, 10634)
X_test_bow  shape: (2000, 10634)


In [8]:
# read in BOW tag model
vectorizer = CountVectorizer(min_df= 1,max_df = 0.50)
tag_test_bow = vectorizer.fit_transform([sentence_filter(d) for d in test_tag]).toarray()
tag_train_bow = vectorizer.transform(np.array([sentence_filter(train_dev_tag[i]) for i in split_idx])).toarray()
#train test split
tag_dev_bow =tag_train_bow[8000:10000,:]
tag_train_bow = tag_train_bow [:8000,:]

print("Built all BOW tag!")
print("tag_train_bow shape:", tag_train_bow.shape)
print("tag_dev_bow shape:", tag_dev_bow.shape)
print("tag_test_bow  shape:",tag_test_bow .shape)

Built all BOW tag!
tag_train_bow shape: (8000, 93)
tag_dev_bow shape: (2000, 93)
tag_test_bow  shape: (2000, 93)


In [11]:
# read in baseline provided resnet1000
y_train_dev = parse_features("features_train/features_resnet1000_train.csv") 
y_train = y_train_dev[split_idx[:num_train]]
y_dev = y_train_dev[split_idx[num_train:]]
y_test = parse_features("features_test/features_resnet1000_test.csv") 

print("Built all resnet ori 1000!")
print("y_train shape:", y_train.shape)
print("y_dev shape:", y_dev.shape)
print("y_test shape:", y_test.shape)

Built all resnet ori 1000!
y_train shape: (8000, 1000)
y_dev shape: (2000, 1000)
y_test shape: (2000, 1000)


In [12]:
# read in baseline provided resnet2048
y2_train_dev = parse_features("features_train/features_resnet1000intermediate_train.csv") 
y2_train = y2_train_dev[split_idx[:num_train]]
y2_dev = y2_train_dev[split_idx[num_train:]]
y2_test = parse_features("features_test/features_resnet1000intermediate_test.csv")

print("Built all resnet ori 2048!")
print("y2_train shape:", y2_train.shape)
print("y2_dev shape:", y2_dev.shape)
print("y2_test shape:", y2_test.shape)

Built all resnet ori 2048!
y2_train shape: (8000, 2048)
y2_dev shape: (2000, 2048)
y2_test shape: (2000, 2048)


#### [caution]
- The following files can be generated by running the feature generating code in the other notebook or download from https://drive.google.com/drive/folders/1pkXLFcvuEkC_VQ-QunVUC9EjjlvF8-j5?usp=sharing (generated_resenet_features.zip)

In [20]:
# read in self generated reset feature
resnet50_train = pd.read_csv('resnet50_train.csv')
resnet101_train = pd.read_csv('resnet101_train.csv')
resnet152_train = pd.read_csv('resnet152_train.csv')
resnetnext_train = pd.read_csv('resnet_ResNext_train.csv')
resnetwide_train = pd.read_csv('resnet_wide101_train.csv')

resnet50_test = pd.read_csv('resnet50_test.csv')
resnet101_test = pd.read_csv('resnet101_test.csv')
resnet152_test = pd.read_csv('resnet152_test.csv')
resnetnext_test = pd.read_csv('resnet_ResNext_test.csv')
resnetwide_test = pd.read_csv('resnet_wide101_test.csv')

#merge data
y_train_allpic_bowcaptag_s = np.hstack([y_train,y2_train,\
                      resnet50_train[[str(i) for i in range(3048)]].values[:8000,:],\
                     resnet101_train[[str(i) for i in range(3048)]].values[:8000,:],\
                      resnet152_train[[str(i) for i in range(3048)]].values[:8000,:],\
                    resnetnext_train[[str(i) for i in range(3048)]].values[:8000,:],\
                      resnetwide_train[[str(i) for i in range(3048)]].values[:8000,:],\
                                      tag_train_bow ])

y_dev_allpic_bowcaptag_s = np.hstack([y_dev,y2_dev,\
                      resnet50_train[[str(i) for i in range(3048)]].values[8000:,:],\
                     resnet101_train[[str(i) for i in range(3048)]].values[8000:,:],\
                      resnet152_train[[str(i) for i in range(3048)]].values[8000:,:],\
                    resnetnext_train[[str(i) for i in range(3048)]].values[8000:,:],\
                      resnetwide_train[[str(i) for i in range(3048)]].values[8000:,:],\
                                    tag_dev_bow])

y_test_allpic_bowcaptag_s = np.hstack([y_test,y2_test,\
                      resnet50_test[[str(i) for i in range(3048)]].values,\
                     resnet101_test[[str(i) for i in range(3048)]].values,\
                      resnet152_test[[str(i) for i in range(3048)]].values,\
                    resnetnext_test[[str(i) for i in range(3048)]].values,\
                      resnetwide_test[[str(i) for i in range(3048)]].values,\
                                     tag_test_bow])

## Best performer
### 4 kernel ridge regressions ensmeble on augemented Resnet features + Tag BoW

#### 1.1 Word2Vec Kernel Ridge(Y= w2v des) - rbf kernel

In [23]:

x_train_all = np.concatenate([x_train_vec, x_dev_vec])
y_train_all_pic_bowcaptag_s = np.concatenate([y_train_allpic_bowcaptag_s, y_dev_allpic_bowcaptag_s])

parameters = {"alpha": [0.01,0.1,1,2,3,5,10],'gamma':[0.01,0.1, 1], "kernel":['rbf']}
reg_vec_k_s = GridSearchCV(KernelRidge(), parameters, cv=5,n_jobs=-1)
print (x_train_all.shape)
print (y_train_all_pic_bowcaptag_s.shape)
reg_vec_k_s.fit(y_train_all_pic_bowcaptag_s,x_train_all)
print (reg_vec_k_s.best_estimator_)

x_test_pred_rid_vec_2_k_s = reg_vec_k_s.predict(y_test_allpic_bowcaptag_s)
test_distances_rid_vec_2_k_s = dist_matrix(x_test_pred_rid_vec_2_k_s , x_test_vec).T

(10000, 300)
(10000, 18381)
KernelRidge(alpha=1, coef0=1, degree=3, gamma=None, kernel='rbf',
      kernel_params=None)


#### 1.2 Word2Vec Kernel Ridge(Y= w2v des) - ploynomial kernel

In [30]:
x_train_all = np.concatenate([x_train_vec, x_dev_vec])
y_train_all_pic_bowcaptag_s = np.concatenate([y_train_allpic_bowcaptag_s, y_dev_allpic_bowcaptag_s])

parameters = {"alpha": [1,5,10,20,30,50],'gamma':[0.01,0.1, 1], "kernel":['polynomial']}
reg_vec_p_s = GridSearchCV(KernelRidge(), parameters, cv=5,n_jobs=-1)
print (x_train_all.shape)
print (y_train_all_pic_bowcaptag_s.shape)
reg_vec_p_s.fit(y_train_all_pic_bowcaptag_s,x_train_all)
print (reg_vec_p_s.best_estimator_)

x_test_pred_rid_vec_2_p_s = reg_vec_p_s.predict(y_test_allpic_bowcaptag_s)
test_distances_rid_vec_2_p_s = dist_matrix(x_test_pred_rid_vec_2_p_s , x_test_vec).T

(10000, 300)
(10000, 18381)
KernelRidge(alpha=30, coef0=1, degree=3, gamma=None, kernel='polynomial',
      kernel_params=None)


#### 2.1 Word2Vec kernel Ridge(Y= BOW des)  - rbf kernel

In [37]:
x_train_all_bow = np.concatenate([x_train_bow, x_dev_bow])
y_train_all_pic_bowcaptag_s = np.concatenate([y_train_allpic_bowcaptag_s, y_dev_allpic_bowcaptag_s])
#print ('x_train_all_bow',x_train_all_bow.shape)
#print ('y_train_all_pic_bowcaptag',y_train_all_pic_bowcaptag.shape)
print (x_train_all_bow.shape)
print (y_train_all_pic_bowcaptag_s.shape)

parameters = {"alpha": [0.1,1,2,3,5,10], 'kernel':['rbf'],'gamma':[0.01,0.1, 1]}
reg_k_s = GridSearchCV(KernelRidge(), parameters, cv=5,n_jobs=-1)
reg_k_s.fit(y_train_all_pic_bowcaptag_s,x_train_all_bow)
print (reg_k_s.best_estimator_)
x_test_pred_rid_bow_2_k_s = reg_k_s.predict(y_test_allpic_bowcaptag_s)
test_distances_rid_bow_2_k_s = dist_matrix(x_test_pred_rid_bow_2_k_s, X_test_bow).T

(10000, 10634)
(10000, 18381)
KernelRidge(alpha=2, coef0=1, degree=3, gamma=None, kernel='rbf',
      kernel_params=None)


#### 2.2 Word2Vec kernel Ridge(Y= BOW des)  - polynomial kernel

In [34]:
x_train_all_bow = np.concatenate([x_train_bow, x_dev_bow])
y_train_all_pic_bowcaptag_s = np.concatenate([y_train_allpic_bowcaptag_s, y_dev_allpic_bowcaptag_s])
#print ('x_train_all_bow',x_train_all_bow.shape)
#print ('y_train_all_pic_bowcaptag',y_train_all_pic_bowcaptag.shape)
print (x_train_all_bow.shape)
print (y_train_all_pic_bowcaptag_s.shape)

parameters = {"alpha": [5,20,30,50,80,100,200,500],'gamma':[0.01,0.1, 1], "kernel":['polynomial']}
reg_p_s = GridSearchCV(KernelRidge(), parameters, cv=5,n_jobs=-1)
reg_p_s.fit(y_train_all_pic_bowcaptag_s,x_train_all_bow)
print (reg_p_s.best_estimator_)
x_test_pred_rid_bow_2_p_s = reg_p_s.predict(y_test_allpic_bowcaptag_s)
test_distances_rid_bow_2_p_s = dist_matrix(x_test_pred_rid_bow_2_p_s, X_test_bow).T

(10000, 10634)
(10000, 18381)
KernelRidge(alpha=50, coef0=1, degree=3, gamma=None, kernel='polynomial',
      kernel_params=None)


#### 3 output

In [56]:
test_distances_total = test_distances_rid_vec_2_k_s/test_distances_rid_vec_2_k_s.mean()\
        +  test_distances_rid_bow_2_k_s/test_distances_rid_bow_2_k_s.mean() + \
 test_distances_rid_vec_2_p_s/test_distances_rid_vec_2_p_s.mean()\
        +  test_distances_rid_bow_2_p_s/test_distances_rid_bow_2_p_s.mean() 

pred_rows = []
for i in range(num_test):
    test_dist_idx = list(np.argsort(test_distances_total[i]))
    top_20 = test_dist_idx[:20]
    row = ["%d.jpg" % i for i in test_dist_idx[:20]]
    pred_rows.append(" ".join(row))

with open("4KernelRidge_best_model.csv", "w") as f:
    f.write("Descritpion_ID,Top_20_Image_IDs\n")
    for i, row in enumerate(pred_rows):
        f.write("%d.txt,%s\n" % (i, row))
print("Output written!")

Output written!


### Other tried models,  only for reference 
#### Model 1
- X: resnet augmented features (3048*4) + Tags(300)  
- 3 models ensemble(1.ridge (Y=BOW des) + 2. ridge(Y= w2v des) + 3. GBM (Y= w2v des)) 

#### 1. Word2Vec Ridge(Y= w2v des)

In [ ]:
x_train_all = np.concatenate([x_train_vec, x_dev_vec])
y_train_all_pic_vectag = np.concatenate([y_train_allpic_vectag, y_dev_allpic_vectag])

parameters = {"alpha": [30000,40000,50000]}
reg_ = GridSearchCV(Ridge(), parameters, cv=5,n_jobs=-1)
reg_.fit(y_train_all_pic_vectag,x_train_all)
print (reg_.best_estimator_)
x_test_pred_rid_vec = reg_.predict(y_test_allpic_vectag)
test_distances_rid_vec = dist_matrix(x_test_pred_rid_vec, x_test_vec).T

#### 2. Word2Vec Ridge(Y= BOW des)

In [ ]:
x_train_all_bow = np.concatenate([x_train_bow, x_dev_bow])
y_train_all_pic_vectag = np.concatenate([y_train_allpic_vectag, y_dev_allpic_vectag])
print ('x_train_all_bow',x_train_all_bow.shape)
print ('y_train_all_pic_vectag',y_train_all_pic_vectag.shape)

parameters = {"alpha": [30000,40000,50000,60000,70000,80000]}
reg_bow = GridSearchCV(Ridge(), parameters, cv=5,n_jobs=-1)
reg_bow.fit(y_train_all_pic_vectag, x_train_all_bow)
print (reg_bow.best_estimator_)
x_test_pred_rid_bow = reg_bow.predict(y_test_allpic_vectag)
test_distances_rid_bow = dist_matrix(x_test_pred_rid_bow, X_test_bow).T

#### 3 GBM (Y= w2v des) 

In [ ]:
x_train_all = np.concatenate([x_train_vec, x_dev_vec])
y_train_all_pic_vectag = np.concatenate([y_train_allpic_vectag, y_dev_allpic_vectag])

#### [caution]
- The following models can be generated by running the code in the next part or download from https://drive.google.com/drive/folders/1pkXLFcvuEkC_VQ-QunVUC9EjjlvF8-j5?usp=sharing (gbm_models)

In [ ]:
# if choose to retrain the model (time consuming), pls set train = True
train = False
if train:
    for i in range (300):
        gbm = lgb.LGBMRegressor(n_estimators=100, n_jobs=-1).fit(y_train_all, x_train_all[:,i])
        gbm.fit(y_train_all_pic_vectag, x_train_all[:,i])
        joblib.dump(gbm, "./gbm_model/t_gbm_all_feature_model_%s.pkl"%str(i))
        del gbm 
        gc.collect()  
    model_list = [joblib.load("./gbm_model/gbm_all_feature_model_%s.pkl"%str(i)) for i in range(300)]
    x_test_pred_gbm_vec = np.array([i.predict(y_test_allpic_vectag) for i in model_list ]).T
    test_distances_gbm_vec = dist_matrix(x_test_pred, x_test).T
else:
    model_list = [joblib.load("./gbm_model/gbm_all_feature_model_%s.pkl"%str(i)) for i in range(300)]
    x_test_pred_gbm_vec = np.array([i.predict(y_test_allpic_vectag) for i in model_list ]).T
    test_distances_gbm_vec= dist_matrix(x_test_pred_gbm_vec, x_test_vec).T

#### output model 1

In [ ]:
#ensemble results from models 
test_distances_total = test_distances_rid_vec + test_distances_rid_bow + test_distances_gbm_vec

#output
pred_rows = []
for i in range(num_test):
    test_dist_idx = list(np.argsort(test_distances_total[i]))
    top_20 = test_dist_idx[:20]
    row = ["%d.jpg" % i for i in test_dist_idx[:20]]
    pred_rows.append(" ".join(row))

with open("other_model_1.csv", "w") as f:
    f.write("Descritpion_ID,Top_20_Image_IDs\n")
    for i, row in enumerate(pred_rows):
        f.write("%d.txt,%s\n" % (i, row))

print("Output written!")

#### Model 2

- X: resnet augmented features (3048*4) + BOW Tags + BOW captions
- 3 model ensemble(1. ridge(Y=BOW des, X as described above) + 2. Word2Vec Ridge(Y=w2v des,X as described above) + 3. GBM (Y= w2v des, X= resnet augmented features (3048*4) + Tag Vec)

In [ ]:
x_train_bow_vec = np.hstack([x_train_bow, x_train_vec])
x_test_bow_vec = np.hstack([X_test_bow, x_test_vec])
x_dev_bow_vec = np.hstack([x_dev_bow, x_dev_vec])

In [ ]:
x_test_bow_vec = np.hstack([X_test_bow, x_test_vec])
x_test_bow_vec.shape

#### 1. Word2Vec Ridge(Y= w2v des)

In [ ]:
x_train_all = np.concatenate([x_train_vec, x_dev_vec])
y_train_all_pic_bowcaptag = np.concatenate([y_train_allpic_bowcaptag, y_dev_allpic_bowcaptag])
#print ('x_train_all_bow',x_train_all.shape)
#print ('y_train_all_pic_bowcaptag',y_train_all_pic_bowcaptag.shape)

parameters = {"alpha": [30000,40000,50000]}
reg_vec = GridSearchCV(Ridge(), parameters, cv=5,n_jobs=-1)
reg_vec.fit(y_train_all_pic_bowcaptag,x_train_all)
#print (reg_vec.best_estimator_)

x_test_pred_rid_vec_2 = reg_vec.predict(y_test_allpic_bowcaptag)
test_distances_rid_vec_2 = dist_matrix(x_test_pred_rid_vec_2 , x_test_vec).T

#### 2. Word2Vec Ridge(Y= BOW des)

In [ ]:
x_train_all_bow = np.concatenate([x_train_bow, x_dev_bow])
y_train_all_pic_bowcaptag = np.concatenate([y_train_allpic_bowcaptag, y_dev_allpic_bowcaptag])
#print ('x_train_all_bow',x_train_all_bow.shape)
#print ('y_train_all_pic_bowcaptag',y_train_all_pic_bowcaptag.shape)

parameters = {"alpha": [30000,40000,50000,60000,70000]}
reg = GridSearchCV(Ridge(), parameters, cv=5,n_jobs=-1)
reg.fit(y_train_all_pic_bowcaptag,x_train_all_bow)
#print (reg.best_estimator_)
x_test_pred_rid_bow_2 = reg.predict(y_test_allpic_bowcaptag)
test_distances_rid_bow_2 = dist_matrix(x_test_pred_rid_bow_2, X_test_bow).T

#### 3 GBM (Y= w2v des) 
- use previous one

#### output model result

In [ ]:
#ensemble results for model 9
test_distances_total = test_distances_rid_vec_2 + test_distances_rid_bow_2 + test_distances_gbm_vec

#output
pred_rows = []
for i in range(num_test):
    test_dist_idx = list(np.argsort(test_distances_total[i]))
    top_20 = test_dist_idx[:20]
    row = ["%d.jpg" % i for i in test_dist_idx[:20]]
    pred_rows.append(" ".join(row))

with open("other_model_2.csv", "w") as f:
    f.write("Descritpion_ID,Top_20_Image_IDs\n")
    for i, row in enumerate(pred_rows):
        f.write("%d.txt,%s\n" % (i, row))
print("Output written!")